In [ ]:
from typing import Annotated, TypedDict

from dotenv import load_dotenv
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_openai import ChatOpenAI
from langgraph.graph.message import add_messages
#from pydantic import BaseModel
from langgraph.graph import StateGraph, START, END
# add memory in the RAM
from langgraph.checkpoint.memory import MemorySaver


load_dotenv()

In [2]:
class ChatState(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]

In [3]:
def chat_node(state: ChatState) -> ChatState:
    llm = ChatOpenAI()
    messages = llm.invoke(state["messages"])
    return {"messages": [messages]}


In [ ]:
model = StateGraph(ChatState)
model.add_node('chat_node', chat_node)
model.add_edge(START, 'chat_node')
model.add_edge('chat_node', END)

graph = model.compile(checkpointer=MemorySaver())

graph



In [6]:
init_state = {'messages': [HumanMessage(content = 'What is the capital of Delhi')]}
thread_id = 1

config = {'configurable': {'thread_id': thread_id}}
result = graph.invoke(init_state, config = config)

result['messages'][-1].content


'The capital of Delhi is New Delhi.'

In [ ]:
#make the chatbot interface in terminal
thread_id = 1
config = {'configurable': {'thread_id': thread_id}}

while True:
    user_input = input("Type here: ")
    print("You:", user_input)
    if user_input.strip().lower() in ('quit', 'exit', 'q', 'bye'):
        break

    init_state = {'messages': [HumanMessage(content = user_input)]}
    result = graph.invoke(init_state, config = config)
    print('AI: ', result['messages'][-1].content)
